In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Electra 모델 클래스
class ElectraVectorizer:
    def __init__(self, model_name):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name).to(self.device)

    def get_first_hidden_state(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(self.device)
        #tokens = vectorizer.tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
        #print("Tokens:", tokens)
        #print("Token IDs:", inputs['input_ids'][0])
        with torch.no_grad():
            outputs = self.model(**inputs)
            #print(outputs.last_hidden_state[0].size())
        return outputs.last_hidden_state[0][0].cpu().numpy()  # 첫 번째 hidden state 벡터를 사용


In [5]:
# 데이터 로드
books_df4 = pd.read_csv('/content/diary.csv', sep='\t')  # 가사 데이터 로드
book_texts4 = books_df4['diary'].tolist()

# 모델 초기화
model_name = "blue2959/koelectra-base-v3-discriminator-finetuned-kor-8-emotions_v1.4"
vectorizer = ElectraVectorizer(model_name)

# 도서 벡터 미리 계산하여 저장
book_vectors4 = [vectorizer.get_first_hidden_state(text) for text in book_texts4]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/815k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/452M [00:00<?, ?B/s]

In [6]:
# 벡터 형태 출력
for i, vec in enumerate(book_vectors4):
    print(f"Vector {i} shape: {vec.shape}")

Vector 0 shape: (768,)
Vector 1 shape: (768,)
Vector 2 shape: (768,)
Vector 3 shape: (768,)
Vector 4 shape: (768,)
Vector 5 shape: (768,)
Vector 6 shape: (768,)
Vector 7 shape: (768,)
Vector 8 shape: (768,)
Vector 9 shape: (768,)
Vector 10 shape: (768,)
Vector 11 shape: (768,)
Vector 12 shape: (768,)
Vector 13 shape: (768,)
Vector 14 shape: (768,)
Vector 15 shape: (768,)


In [7]:
import numpy as np

# book_vectors4 리스트를 NumPy 배열로 변환
book_vectors_matrix = np.array(book_vectors4)

# 결과 출력
print(book_vectors_matrix.shape)

(16, 768)


In [8]:
import numpy as np
import pickle

# NumPy 배열을 .npy 파일로 저장
book_vectors_matrix = np.array(book_vectors4)
np.save('book_vectors_matrix.npy', book_vectors_matrix)

# NumPy 배열을 pickle 파일로 저장
with open('diary_matrix.pkl', 'wb') as f:
    pickle.dump(book_vectors_matrix, f)

print("NumPy 배열이 pickle 파일로 저장되었습니다.")

NumPy 배열이 pickle 파일로 저장되었습니다.
